# Project3: Twitter Sentiment Analysis

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
#necessary common imports:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [3]:
# For text preprocessing:
import re #regex module
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
import string

# Required downloadable packages:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\johnd\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\johnd\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\johnd\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Reading the Data from the Given excel file:

In [4]:
# Loading the data:
data = pd.read_csv('Twitter_Data.csv')
data.head()

,clean_text,category
0,when modi promised “minimum government maximum...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162980 entries, 0 to 162979
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   clean_text  162976 non-null  object 
 1   category    162973 non-null  float64
dtypes: float64(1), object(1)
memory usage: 2.5+ MB


In [6]:
data.shape

(162980, 2)

## Changing our dependent variable into categorical. (0 to “Neutral,”-1 to “Negative”, 1 to “Positive”):

In [7]:
data.category.value_counts()

 1.0    72250
 0.0    55213
-1.0    35510
Name: category, dtype: int64

In [8]:
mapping = {0:"Neutral", 1:"Positive",-1:"Negative"}

In [9]:
data['category'] = data['category'].replace(mapping)

In [10]:
data.category.value_counts()

Positive    72250
Neutral     55213
Negative    35510
Name: category, dtype: int64

## Performing missing value analysis and dropping all null/missing values:

In [11]:
data.isna().sum() #reviewing null values

clean_text    4
category      7
dtype: int64

In [12]:
data.dropna(inplace=True)

In [13]:
data.isna().sum() #again reviewing, no null values present.

clean_text    0
category      0
dtype: int64

In [14]:
data.shape

(162969, 2)

In [15]:
#renaming the column name to 'Tweets'
data = data.rename(columns={'clean_text': 'Tweets'})

## Performing text cleaning. (removing every symbol except alphanumeric, transforming all words to lower case, and removing punctuation and stopwords ):

In [16]:
# For text-preprocessing:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
import string

# For model building:
from keras.models import Sequential
from keras.layers import Input, Dense, Embedding, LSTM, Dropout

punct = string.punctuation #storing all punctuations
punct

In [17]:
# Stopwords list:
stopWords = stopwords.words('english')
print(stopWords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [18]:
## Reviewing the first record:
#data['clean_text'][0]
data['Tweets'][0]

'when modi promised “minimum government maximum governance” expected him begin the difficult job reforming the state why does take years get justice state should and not business and should exit psus and temples'

#### Text-preprocessing steps:

In [19]:
def cleaning(text):
    # Removes all non-alphabetic characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Convert to lowercase
    text = text.lower()
    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [20]:
#data['clean_text'] = data['clean_text'].apply(cleaning)
data['Tweets'] = data['Tweets'].apply(cleaning)

In [21]:
#data['clean_text'][0]
data['Tweets'][0]

'when modi promised minimum government maximum governance expected him begin the difficult job reforming the state why does take years get justice state should and not business and should exit psus and temples'

In [22]:
def processing_text(text):
    # Tokenize the text
    tokens = word_tokenize(text)
    
    # Remove stopwords
    stop_words = set(stopWords)
    filtered_tokens = [token for token in tokens if token not in stop_words]
    
    # Apply stemming
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(token) for token in filtered_tokens]
    
    # Apply lemmatization
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in stemmed_tokens]
    
    # Join the tokens back into a string and return it
    return ' '.join(lemmatized_tokens)

In [23]:
#data['clean_text'] = data['clean_text'].apply(processing_text)
data['Tweets'] = data['Tweets'].apply(processing_text)

In [24]:
# data['clean_text'][0]
data['Tweets'][0]

'modi promis minimum govern maximum govern expect begin difficult job reform state take year get justic state busi exit psu templ'

## Splitting data into dependent(Y) and independent(X) sets:

In [25]:
X = data.Tweets
Y = data.category

In [26]:
sequences = X.copy()

## Performing operations on text data:

### * Performing one-hot encoding for each sentence(using TensorFlow):

In [27]:
from keras.preprocessing.text import one_hot
one_hot_encoded = []

for word in sequences:
    vocab_size = 2000
    encoded_words = one_hot(word, vocab_size)
    one_hot_encoded.append(encoded_words)

In [28]:
print(one_hot_encoded[0])

[1001, 1495, 1197, 296, 1257, 296, 738, 431, 142, 1778, 1930, 1429, 1718, 851, 342, 1950, 1429, 424, 377, 558, 1849]


### * Adding padding from the front side (using Tensorflow):

In [29]:
from keras.preprocessing.sequence import pad_sequences
max_length =  30
seq_mat = pad_sequences(one_hot_encoded, maxlen=max_length)
#seq_mat

In [30]:
seq_mat[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0, 1001, 1495,
       1197,  296, 1257,  296,  738,  431,  142, 1778, 1930, 1429, 1718,
        851,  342, 1950, 1429,  424,  377,  558, 1849])

### * Building an LSTM model and compiling it(describing features, input length,vocabulary size, information drop-out layer, activation function for output)

In [31]:
## This is the finalized model I'm gonna use for the project......
from keras.layers import Input, Dense, Embedding, LSTM, Dropout
from keras.models import Sequential, Model

model = Sequential() #step1: instantiating the model function
model.add(Input(30,)) # step2: adding input layer of 30 input size
model.add(Embedding(input_dim=vocab_size, output_dim=32)) # step3: adding embedding layer, giving input dim, output dim
model.add(LSTM(units=64)) #step4: adding LSTM layer with 64 neurons
model.add(Dropout(rate=0.5)) #step5: dropping some neurons
model.add(Dense(units=3, activation='sigmoid')) #step6: output layer, giving out three classification output

In [32]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 30, 32)            64000     
                                                                 
 lstm (LSTM)                 (None, 64)                24832     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 3)                 195       
                                                                 
Total params: 89027 (347.76 KB)
Trainable params: 89027 (347.76 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


vocab_size * output_dim + (4 * 64 * (64 + 30 + 1)) + (64 * 30) + 64 + (3 * 64) + 3

In [33]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [34]:
X_final = seq_mat #creating Final independent variable
#Y = data.category

### * Performing dummy variable creation for the dependent variable:

In [35]:
Y_final = pd.get_dummies(Y) #creating dummy variables/columns from Y
Y_final

,Negative,Neutral,Positive
0,1,0,0
1,0,1,0
2,0,0,1
3,0,0,1
4,0,0,1
...,...,...,...
162975,1,0,0
162976,1,0,0
162977,0,1,0
162978,0,1,0


### * Splitting the data into tests and train:

In [36]:
#splitting into 70%-30% ratio
X_train, X_test, Y_train, Y_test = train_test_split(X_final, Y_final, test_size=0.3) 

## Training new model:

    model.fit(X_train, Y_train)
    3565/3565 [==============================] - 160s 43ms/step - loss: 0.4360 - accuracy: 0.6980
    <keras.src.callbacks.History at 0x177325866e0>

In [37]:
# Fitting training set and giving validation_split of 10%:
model.fit(X_train, Y_train, batch_size=80, epochs=10,  validation_split=0.1)

Epoch 1/10
1284/1284 [==============================] - 88s 64ms/step - loss: 0.4514 - accuracy: 0.6808 - val_loss: 0.4058 - val_accuracy: 0.7227
Epoch 2/10
1284/1284 [==============================] - 81s 63ms/step - loss: 0.3963 - accuracy: 0.7346 - val_loss: 0.3960 - val_accuracy: 0.7266
Epoch 3/10
1284/1284 [==============================] - 89s 70ms/step - loss: 0.3872 - accuracy: 0.7397 - val_loss: 0.3941 - val_accuracy: 0.7273
Epoch 4/10
1284/1284 [==============================] - 84s 65ms/step - loss: 0.3787 - accuracy: 0.7444 - val_loss: 0.3923 - val_accuracy: 0.7282
Epoch 5/10
1284/1284 [==============================] - 86s 67ms/step - loss: 0.3703 - accuracy: 0.7498 - val_loss: 0.3932 - val_accuracy: 0.7261
Epoch 6/10
1284/1284 [==============================] - 105s 82ms/step - loss: 0.3634 - accuracy: 0.7562 - val_loss: 0.3949 - val_accuracy: 0.7347
Epoch 7/10
1284/1284 [==============================] - 97s 75ms/step - loss: 0.3566 - accuracy: 0.7617 - val_loss: 0.3956 

In [38]:
#Evaluating our model by providing test data:
model.evaluate(X_test, Y_test)

1528/1528 [==============================] - 23s 15ms/step - loss: 0.4028 - accuracy: 0.7354


[0.40280699729919434, 0.7353705167770386]

In [39]:
# Generating predictions:
acc = model.predict(X_test)

1528/1528 [==============================] - 24s 15ms/step


In [40]:
#Checking its type:
type(acc)

numpy.ndarray

In [41]:
# Reviewing first 10 records:
acc[:10]

array([[0.00459016, 0.00321639, 0.99394584],
       [0.1358922 , 0.07370602, 0.76688546],
       [0.00224614, 0.98353314, 0.0236471 ],
       [0.2235105 , 0.04992226, 0.7181809 ],
       [0.9159823 , 0.10393535, 0.01837717],
       [0.43619904, 0.3805365 , 0.20339786],
       [0.21304017, 0.42884645, 0.21677402],
       [0.09843527, 0.20717186, 0.6802411 ],
       [0.74473965, 0.14254755, 0.07890027],
       [0.22839625, 0.09501585, 0.6831377 ]], dtype=float32)

In [42]:
pred = acc.copy() #copying
pred[0] #first record

array([0.00459016, 0.00321639, 0.99394584], dtype=float32)

## Normalizing the predictions as same as the original data (predictions might be in decimal, so whatsoever is nearest to 1 is predicted as yes and set other as 0):

In [43]:
count = 0
for i in range(len(pred)):
    if (pred[i][0] > pred[i][1] and pred[i][0] > pred[i][2]):
        count = 0
    elif (pred[i][1] > pred[i][0] and pred[i][1] > pred[i][2]):
        count = 1
    elif (pred[i][2] > pred[i][0] and pred[i][2] > pred[i][1]):
        count = 2
    print(i,"for count is",count)   
    for j in range(3):
        pred[i][j] = 0   
        pred[i][count] = 1
    print(pred[i])
    count = 0

0 for count is 2
[0. 0. 1.]
1 for count is 2
[0. 0. 1.]
2 for count is 1
[0. 1. 0.]
3 for count is 2
[0. 0. 1.]
4 for count is 0
[1. 0. 0.]
5 for count is 0
[1. 0. 0.]
6 for count is 1
[0. 1. 0.]
7 for count is 2
[0. 0. 1.]
8 for count is 0
[1. 0. 0.]
9 for count is 2
[0. 0. 1.]
10 for count is 1
[0. 1. 0.]
11 for count is 0
[1. 0. 0.]
12 for count is 2
[0. 0. 1.]
13 for count is 2
[0. 0. 1.]
14 for count is 1
[0. 1. 0.]
15 for count is 2
[0. 0. 1.]
16 for count is 1
[0. 1. 0.]
17 for count is 0
[1. 0. 0.]
18 for count is 2
[0. 0. 1.]
19 for count is 2
[0. 0. 1.]
20 for count is 2
[0. 0. 1.]
21 for count is 1
[0. 1. 0.]
22 for count is 1
[0. 1. 0.]
23 for count is 0
[1. 0. 0.]
24 for count is 2
[0. 0. 1.]
25 for count is 1
[0. 1. 0.]
26 for count is 2
[0. 0. 1.]
27 for count is 2
[0. 0. 1.]
28 for count is 0
[1. 0. 0.]
29 for count is 2
[0. 0. 1.]
30 for count is 2
[0. 0. 1.]
31 for count is 2
[0. 0. 1.]
32 for count is 0
[1. 0. 0.]
33 for count is 2
[0. 0. 1.]
34 for count is 1
[0. 1.

[0. 1. 0.]
673 for count is 2
[0. 0. 1.]
674 for count is 1
[0. 1. 0.]
675 for count is 2
[0. 0. 1.]
676 for count is 2
[0. 0. 1.]
677 for count is 2
[0. 0. 1.]
678 for count is 2
[0. 0. 1.]
679 for count is 2
[0. 0. 1.]
680 for count is 0
[1. 0. 0.]
681 for count is 1
[0. 1. 0.]
682 for count is 2
[0. 0. 1.]
683 for count is 0
[1. 0. 0.]
684 for count is 2
[0. 0. 1.]
685 for count is 2
[0. 0. 1.]
686 for count is 1
[0. 1. 0.]
687 for count is 1
[0. 1. 0.]
688 for count is 0
[1. 0. 0.]
689 for count is 2
[0. 0. 1.]
690 for count is 2
[0. 0. 1.]
691 for count is 2
[0. 0. 1.]
692 for count is 1
[0. 1. 0.]
693 for count is 1
[0. 1. 0.]
694 for count is 2
[0. 0. 1.]
695 for count is 0
[1. 0. 0.]
696 for count is 2
[0. 0. 1.]
697 for count is 2
[0. 0. 1.]
698 for count is 0
[1. 0. 0.]
699 for count is 0
[1. 0. 0.]
700 for count is 1
[0. 1. 0.]
701 for count is 0
[1. 0. 0.]
702 for count is 1
[0. 1. 0.]
703 for count is 0
[1. 0. 0.]
704 for count is 2
[0. 0. 1.]
705 for count is 2
[0. 0. 1.]

[0. 0. 1.]
1438 for count is 0
[1. 0. 0.]
1439 for count is 2
[0. 0. 1.]
1440 for count is 2
[0. 0. 1.]
1441 for count is 1
[0. 1. 0.]
1442 for count is 1
[0. 1. 0.]
1443 for count is 1
[0. 1. 0.]
1444 for count is 2
[0. 0. 1.]
1445 for count is 1
[0. 1. 0.]
1446 for count is 2
[0. 0. 1.]
1447 for count is 2
[0. 0. 1.]
1448 for count is 2
[0. 0. 1.]
1449 for count is 0
[1. 0. 0.]
1450 for count is 0
[1. 0. 0.]
1451 for count is 2
[0. 0. 1.]
1452 for count is 0
[1. 0. 0.]
1453 for count is 1
[0. 1. 0.]
1454 for count is 1
[0. 1. 0.]
1455 for count is 1
[0. 1. 0.]
1456 for count is 0
[1. 0. 0.]
1457 for count is 2
[0. 0. 1.]
1458 for count is 1
[0. 1. 0.]
1459 for count is 0
[1. 0. 0.]
1460 for count is 1
[0. 1. 0.]
1461 for count is 2
[0. 0. 1.]
1462 for count is 2
[0. 0. 1.]
1463 for count is 2
[0. 0. 1.]
1464 for count is 2
[0. 0. 1.]
1465 for count is 2
[0. 0. 1.]
1466 for count is 1
[0. 1. 0.]
1467 for count is 1
[0. 1. 0.]
1468 for count is 1
[0. 1. 0.]
1469 for count is 2
[0. 0. 1

[0. 1. 0.]
2258 for count is 2
[0. 0. 1.]
2259 for count is 1
[0. 1. 0.]
2260 for count is 2
[0. 0. 1.]
2261 for count is 1
[0. 1. 0.]
2262 for count is 2
[0. 0. 1.]
2263 for count is 0
[1. 0. 0.]
2264 for count is 2
[0. 0. 1.]
2265 for count is 1
[0. 1. 0.]
2266 for count is 0
[1. 0. 0.]
2267 for count is 2
[0. 0. 1.]
2268 for count is 1
[0. 1. 0.]
2269 for count is 1
[0. 1. 0.]
2270 for count is 2
[0. 0. 1.]
2271 for count is 1
[0. 1. 0.]
2272 for count is 2
[0. 0. 1.]
2273 for count is 2
[0. 0. 1.]
2274 for count is 1
[0. 1. 0.]
2275 for count is 1
[0. 1. 0.]
2276 for count is 1
[0. 1. 0.]
2277 for count is 0
[1. 0. 0.]
2278 for count is 1
[0. 1. 0.]
2279 for count is 1
[0. 1. 0.]
2280 for count is 2
[0. 0. 1.]
2281 for count is 0
[1. 0. 0.]
2282 for count is 2
[0. 0. 1.]
2283 for count is 2
[0. 0. 1.]
2284 for count is 0
[1. 0. 0.]
2285 for count is 2
[0. 0. 1.]
2286 for count is 2
[0. 0. 1.]
2287 for count is 0
[1. 0. 0.]
2288 for count is 0
[1. 0. 0.]
2289 for count is 0
[1. 0. 0

[0. 0. 1.]
3099 for count is 1
[0. 1. 0.]
3100 for count is 2
[0. 0. 1.]
3101 for count is 2
[0. 0. 1.]
3102 for count is 1
[0. 1. 0.]
3103 for count is 0
[1. 0. 0.]
3104 for count is 1
[0. 1. 0.]
3105 for count is 2
[0. 0. 1.]
3106 for count is 1
[0. 1. 0.]
3107 for count is 1
[0. 1. 0.]
3108 for count is 1
[0. 1. 0.]
3109 for count is 1
[0. 1. 0.]
3110 for count is 2
[0. 0. 1.]
3111 for count is 2
[0. 0. 1.]
3112 for count is 2
[0. 0. 1.]
3113 for count is 2
[0. 0. 1.]
3114 for count is 1
[0. 1. 0.]
3115 for count is 2
[0. 0. 1.]
3116 for count is 1
[0. 1. 0.]
3117 for count is 1
[0. 1. 0.]
3118 for count is 2
[0. 0. 1.]
3119 for count is 2
[0. 0. 1.]
3120 for count is 2
[0. 0. 1.]
3121 for count is 2
[0. 0. 1.]
3122 for count is 2
[0. 0. 1.]
3123 for count is 1
[0. 1. 0.]
3124 for count is 1
[0. 1. 0.]
3125 for count is 2
[0. 0. 1.]
3126 for count is 1
[0. 1. 0.]
3127 for count is 2
[0. 0. 1.]
3128 for count is 2
[0. 0. 1.]
3129 for count is 1
[0. 1. 0.]
3130 for count is 2
[0. 0. 1

...

[0. 0. 1.]
45615 for count is 1
[0. 1. 0.]
45616 for count is 1
[0. 1. 0.]
45617 for count is 1
[0. 1. 0.]
45618 for count is 2
[0. 0. 1.]
45619 for count is 1
[0. 1. 0.]
45620 for count is 1
[0. 1. 0.]
45621 for count is 2
[0. 0. 1.]
45622 for count is 2
[0. 0. 1.]
45623 for count is 2
[0. 0. 1.]
45624 for count is 0
[1. 0. 0.]
45625 for count is 0
[1. 0. 0.]
45626 for count is 2
[0. 0. 1.]
45627 for count is 1
[0. 1. 0.]
45628 for count is 0
[1. 0. 0.]
45629 for count is 0
[1. 0. 0.]
45630 for count is 2
[0. 0. 1.]
45631 for count is 2
[0. 0. 1.]
45632 for count is 2
[0. 0. 1.]
45633 for count is 0
[1. 0. 0.]
45634 for count is 0
[1. 0. 0.]
45635 for count is 2
[0. 0. 1.]
45636 for count is 2
[0. 0. 1.]
45637 for count is 1
[0. 1. 0.]
45638 for count is 1
[0. 1. 0.]
45639 for count is 1
[0. 1. 0.]
45640 for count is 2
[0. 0. 1.]
45641 for count is 1
[0. 1. 0.]
45642 for count is 2
[0. 0. 1.]
45643 for count is 2
[0. 0. 1.]
45644 for count is 2
[0. 0. 1.]
45645 for count is 1
[0. 1. 0

[0. 1. 0.]
46266 for count is 2
[0. 0. 1.]
46267 for count is 1
[0. 1. 0.]
46268 for count is 1
[0. 1. 0.]
46269 for count is 2
[0. 0. 1.]
46270 for count is 1
[0. 1. 0.]
46271 for count is 2
[0. 0. 1.]
46272 for count is 2
[0. 0. 1.]
46273 for count is 1
[0. 1. 0.]
46274 for count is 0
[1. 0. 0.]
46275 for count is 2
[0. 0. 1.]
46276 for count is 1
[0. 1. 0.]
46277 for count is 0
[1. 0. 0.]
46278 for count is 0
[1. 0. 0.]
46279 for count is 2
[0. 0. 1.]
46280 for count is 2
[0. 0. 1.]
46281 for count is 2
[0. 0. 1.]
46282 for count is 2
[0. 0. 1.]
46283 for count is 0
[1. 0. 0.]
46284 for count is 1
[0. 1. 0.]
46285 for count is 1
[0. 1. 0.]
46286 for count is 0
[1. 0. 0.]
46287 for count is 2
[0. 0. 1.]
46288 for count is 1
[0. 1. 0.]
46289 for count is 1
[0. 1. 0.]
46290 for count is 2
[0. 0. 1.]
46291 for count is 1
[0. 1. 0.]
46292 for count is 1
[0. 1. 0.]
46293 for count is 2
[0. 0. 1.]
46294 for count is 0
[1. 0. 0.]
46295 for count is 1
[0. 1. 0.]
46296 for count is 2
[0. 0. 1

[0. 1. 0.]
46899 for count is 2
[0. 0. 1.]
46900 for count is 1
[0. 1. 0.]
46901 for count is 2
[0. 0. 1.]
46902 for count is 1
[0. 1. 0.]
46903 for count is 1
[0. 1. 0.]
46904 for count is 0
[1. 0. 0.]
46905 for count is 2
[0. 0. 1.]
46906 for count is 1
[0. 1. 0.]
46907 for count is 2
[0. 0. 1.]
46908 for count is 2
[0. 0. 1.]
46909 for count is 2
[0. 0. 1.]
46910 for count is 1
[0. 1. 0.]
46911 for count is 2
[0. 0. 1.]
46912 for count is 0
[1. 0. 0.]
46913 for count is 2
[0. 0. 1.]
46914 for count is 0
[1. 0. 0.]
46915 for count is 1
[0. 1. 0.]
46916 for count is 2
[0. 0. 1.]
46917 for count is 1
[0. 1. 0.]
46918 for count is 0
[1. 0. 0.]
46919 for count is 1
[0. 1. 0.]
46920 for count is 2
[0. 0. 1.]
46921 for count is 1
[0. 1. 0.]
46922 for count is 1
[0. 1. 0.]
46923 for count is 1
[0. 1. 0.]
46924 for count is 2
[0. 0. 1.]
46925 for count is 1
[0. 1. 0.]
46926 for count is 2
[0. 0. 1.]
46927 for count is 2
[0. 0. 1.]
46928 for count is 1
[0. 1. 0.]
46929 for count is 0
[1. 0. 0

[0. 0. 1.]
47594 for count is 1
[0. 1. 0.]
47595 for count is 2
[0. 0. 1.]
47596 for count is 2
[0. 0. 1.]
47597 for count is 2
[0. 0. 1.]
47598 for count is 1
[0. 1. 0.]
47599 for count is 2
[0. 0. 1.]
47600 for count is 0
[1. 0. 0.]
47601 for count is 1
[0. 1. 0.]
47602 for count is 1
[0. 1. 0.]
47603 for count is 0
[1. 0. 0.]
47604 for count is 2
[0. 0. 1.]
47605 for count is 2
[0. 0. 1.]
47606 for count is 1
[0. 1. 0.]
47607 for count is 2
[0. 0. 1.]
47608 for count is 0
[1. 0. 0.]
47609 for count is 1
[0. 1. 0.]
47610 for count is 2
[0. 0. 1.]
47611 for count is 0
[1. 0. 0.]
47612 for count is 2
[0. 0. 1.]
47613 for count is 0
[1. 0. 0.]
47614 for count is 1
[0. 1. 0.]
47615 for count is 1
[0. 1. 0.]
47616 for count is 0
[1. 0. 0.]
47617 for count is 1
[0. 1. 0.]
47618 for count is 1
[0. 1. 0.]
47619 for count is 1
[0. 1. 0.]
47620 for count is 0
[1. 0. 0.]
47621 for count is 2
[0. 0. 1.]
47622 for count is 1
[0. 1. 0.]
47623 for count is 2
[0. 0. 1.]
47624 for count is 2
[0. 0. 1

[0. 0. 1.]
48335 for count is 2
[0. 0. 1.]
48336 for count is 2
[0. 0. 1.]
48337 for count is 2
[0. 0. 1.]
48338 for count is 2
[0. 0. 1.]
48339 for count is 1
[0. 1. 0.]
48340 for count is 2
[0. 0. 1.]
48341 for count is 2
[0. 0. 1.]
48342 for count is 2
[0. 0. 1.]
48343 for count is 2
[0. 0. 1.]
48344 for count is 2
[0. 0. 1.]
48345 for count is 2
[0. 0. 1.]
48346 for count is 0
[1. 0. 0.]
48347 for count is 1
[0. 1. 0.]
48348 for count is 0
[1. 0. 0.]
48349 for count is 0
[1. 0. 0.]
48350 for count is 2
[0. 0. 1.]
48351 for count is 2
[0. 0. 1.]
48352 for count is 1
[0. 1. 0.]
48353 for count is 0
[1. 0. 0.]
48354 for count is 1
[0. 1. 0.]
48355 for count is 0
[1. 0. 0.]
48356 for count is 0
[1. 0. 0.]
48357 for count is 1
[0. 1. 0.]
48358 for count is 2
[0. 0. 1.]
48359 for count is 2
[0. 0. 1.]
48360 for count is 2
[0. 0. 1.]
48361 for count is 1
[0. 1. 0.]
48362 for count is 1
[0. 1. 0.]
48363 for count is 1
[0. 1. 0.]
48364 for count is 1
[0. 1. 0.]
48365 for count is 2
[0. 0. 1

## Testing accuracy:

In [44]:
Y_test

,Negative,Neutral,Positive
148182,0,0,1
93407,0,0,1
41020,0,1,0
33568,0,1,0
50032,1,0,0
...,...,...,...
58996,1,0,0
3755,0,1,0
150686,0,1,0
4987,0,0,1


In [45]:
from sklearn.metrics import accuracy_score
print('Accuracy of the model:', accuracy_score(Y_test, pred))

Accuracy of the model: 0.7353705180912642


## 11. Classification report:

In [46]:
from sklearn.metrics import classification_report
print('Classification report:\n', classification_report(Y_test, pred))


Classification report:
               precision    recall  f1-score   support

           0       0.64      0.60      0.62     10607
           1       0.75      0.77      0.76     16561
           2       0.77      0.78      0.77     21723

   micro avg       0.74      0.74      0.74     48891
   macro avg       0.72      0.71      0.72     48891
weighted avg       0.73      0.74      0.73     48891
 samples avg       0.74      0.74      0.74     48891



Thesis:
    
    1. Performed Data handling by loading the dataset, reviewing its dimensional information, analyzing its data.
    2. Carried-out text-correction by converting text into lower-case, removing all non-alphabetic charaters from it.
    3. Executed text-Preprocessing through stopwords removal, stemming(rooting the word into its root word), lemmatizing(converting to its dictionary form) process.
    4. Built LSTM model for generating classification results and evaluated using test data.
    5. Tested results by performance metrics.